In [1]:
import pandas as pd
import numpy as np

In [2]:
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [3]:
import os

In [4]:
from tqdm import tqdm

In [5]:
import matplotlib.pyplot as plt

<h3>Read raw files</h3>

In [8]:
raw_path = './data/raw/'

In [9]:
raw_data = dd.read_csv(
    raw_path + '*',
    sep=';',  
    encoding='cp1251', 
    low_memory=False, 
    dtype={
        'ГородМагазина': 'object',
        'МагазинЗаказа': 'object',
        'СуммаДоставки': 'object',
        'СуммаУслуг'   : 'object'
    }
)

In [10]:
with ProgressBar():
    data = raw_data.compute()

[########################################] | 100% Completed | 16.5s


In [11]:
data.shape

(1673267, 38)

In [12]:
data.columns

Index(['Дата', 'ДатаДоставки', 'НомерЗаказаНаСайте', 'НовыйСтатус',
       'СуммаЗаказаНаСайте', 'СуммаДокумента', 'МетодДоставки', 'ФормаОплаты',
       'Регион', 'Группа2', 'Группа3', 'Группа4', 'Тип', 'Номенклатура',
       'ТипТовара', 'Отменено', 'ПричинаОтмены', 'Количество', 'Цена',
       'СуммаСтроки', 'ЦенаЗакупки', 'МесяцДатыЗаказа', 'ГодДатыЗаказа',
       'ПВЗ_код', 'Статус', 'Гео', 'Маржа', 'СуммаУслуг', 'СуммаДоставки',
       'НомерСтроки', 'КоличествоПроданоКлиенту', 'ДатаЗаказаНаСайте',
       'Телефон_new', 'ЭлектроннаяПочта_new', 'Клиент', 'ID_SKU',
       'ГородМагазина', 'МагазинЗаказа'],
      dtype='object')

<h3>Clean</h3>

In [13]:
numeric = ['СуммаЗаказаНаСайте', 
           'СуммаДокумента', 
           'Количество', 
           'Цена',
           'СуммаСтроки', 
           'ЦенаЗакупки', 
           'Маржа', 
           'СуммаУслуг',
           'СуммаДоставки', 
           'КоличествоПроданоКлиенту']

def clear_to_float(x):
    return float(str(x).replace(' ', '').replace(',','.'))

for column in numeric:
    data[column] = data[column].apply(clear_to_float)

<h3>Filter data</h3>

In [14]:
assert data['Телефон_new'].isna().sum() == 0

In [15]:
# data with valid user phone number
ph_data = data[data['Телефон_new'].str.contains('[0-9]{8}-[0-9]{14}')] # phone data

In [16]:
ph_data.shape

(1668900, 38)

In [17]:
################################################################################

<h4>Filter region</h4>

In [18]:
TOP_N_REGIONS = 10

regions = ph_data['Регион'].value_counts(normalize=True).round(2)[:TOP_N_REGIONS]
regions

Москва                        0.44
Санкт-Петербург               0.06
Нижний Новгород               0.02
Екатеринбург                  0.02
Люберцы (Люберецкий район)    0.01
Воронеж                       0.01
Самара                        0.01
Ростов-на-Дону                0.01
Краснодар                     0.01
Пермь                         0.01
Name: Регион, dtype: float64

In [19]:
region_data = ph_data[ph_data['Регион'].isin(regions.index)]

In [20]:
################################################################################

<h4>Filter price</h4>

In [21]:
region_data[['Цена', 'СуммаЗаказаНаСайте', 'Количество']].describe().round(2)

,Цена,СуммаЗаказаНаСайте,Количество
count,1023447.00,1023447.00,1023447.00
mean,644.12,4844.12,1.44
std,1665.70,6540.59,2.34
min,0.00,1.00,0.00
25%,79.00,1774.00,1.00
50%,180.00,3111.00,1.00
75%,599.00,5449.00,1.00
max,107000.00,299309.00,387.00


In [22]:
# filter by quantity
pf_data = region_data[region_data['Количество'] != 0]
# filter by price
threshold = pf_data['Цена'].quantile(0.025)
pf_data = pf_data[pf_data['Цена'] > threshold]
print('Remove every order with price lower than', threshold)

Remove every order with price lower than 23.0


In [23]:
pf_data.shape # price filtered data

(963061, 38)

In [24]:
################################################################################

<h4>Filter date</h4>

In [25]:
pf_data['Дата'] = pd.to_datetime(pf_data['Дата'], dayfirst=True)
pf_data['ДатаДоставки'] = pd.to_datetime(pf_data['ДатаДоставки'], dayfirst=True)

In [26]:
print(pf_data['Дата'].min().date().strftime('%d/%m/%Y'), '->', pf_data['Дата'].max().date().strftime('%d/%m/%Y'))

01/07/2017 -> 31/10/2017


In [27]:
print(pf_data['ДатаДоставки'].min().date().strftime('%d/%m/%Y'), '->', pf_data['ДатаДоставки'].max().date().strftime('%d/%m/%Y'))

26/10/2016 -> 29/11/2017


In [28]:
pf_data.groupby(pd.Grouper(key='ДатаДоставки', freq='M'))['Дата'].count()

ДатаДоставки
2016-10-31         2
2016-11-30         0
2016-12-31         0
2017-01-31         0
2017-02-28         0
2017-03-31         0
2017-04-30         0
2017-05-31         0
2017-06-30        19
2017-07-31    177839
2017-08-31    232109
2017-09-30    233585
2017-10-31    266613
2017-11-30     52818
Name: Дата, dtype: int64

In [29]:
# time filtered data
tf_data = pf_data[pf_data['ДатаДоставки'] > pd.Timestamp(year=2017, month=7, day=1)]

In [30]:
################################################################################

<h3>Item parameters</h3>

In [31]:
tf_data['Группа2'].value_counts().head(4)

ТЕКСТИЛЬ, ТРИКОТАЖ    160896
ИГРУШКИ               150241
ДЕТСКОЕ ПИТАНИЕ       147477
КОСМЕТИКА/ГИГИЕНА      84881
Name: Группа2, dtype: int64

In [32]:
tf_data['Группа3'].value_counts().head(4)

ПЮРЕ                                  81267
ПОДГУЗНИКИ                            67372
ИГРУШКИ ДЛЯ РАЗВИТИЯ МАЛЫШЕЙ          46069
ОДЕЖДА ДЛЯ НОВОРОЖДЕННЫХ (0-2 лет)    40362
Name: Группа3, dtype: int64

In [33]:
tf_data['Группа4'].value_counts().head(4)

трусики-подгузники    33687
подгузники            30505
пюре фруктовые        26475
пюре мясные           24653
Name: Группа4, dtype: int64

In [34]:
tf_data['Тип'].value_counts()

ППКП       349788
ИНОЕ       200781
ОДЕЖДА     200507
ИГРУШКИ    183316
КГТ         28572
Name: Тип, dtype: int64

In [35]:
tf_data['ТипТовара'].value_counts()

МГТ    747233
КГТ     28494
Name: ТипТовара, dtype: int64

In [36]:
(tf_data['ID_SKU'].value_counts() > 70).sum()

1392

<h3>DataFrame</h3>

In [42]:
from pathlib import Path

In [43]:
Path("./data/clean").mkdir(parents=True, exist_ok=True)

In [37]:
df = tf_data[['Дата', 'ID_SKU', 'Телефон_new', 'Цена', 'Количество', 'Группа2', 'Группа3', 'Группа4', 'Тип', 'ТипТовара']]

In [38]:
df.dtypes

Дата           datetime64[ns]
ID_SKU                 object
Телефон_new            object
Цена                  float64
Количество            float64
Группа2                object
Группа3                object
Группа4                object
Тип                    object
ТипТовара              object
dtype: object

In [39]:
df.to_parquet('./data/clean/positions.parquet')

In [40]:
################################################################################

In [41]:
tf_data.to_parquet('./data/clean/all_positions.parquet')